In [ ]:
import numpy as np
import random
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
def generate_password(strength):
    if strength == "weak":
        return ''.join(random.choices(string.ascii_lowercase, k=random.randint(4, 6)))
    elif strength == "medium":
        return ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(6, 10)))
    elif strength == "strong":
        return ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=random.randint(10, 16)))
